In [3]:
conda install -c conda-forge featuretools


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\tejal\anaconda3

  added / updated specs:
    - featuretools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.14.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    featuretools-0.27.1        |     pyhd3eb1b0_2         435 KB
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INSTALLED:

  featuretools       pkgs/main/noarch::featuretools-0.27.1-pyhd3eb1b0_2

The following packages will be UPDATED:

  conda                               4.13.0-py39hcbf5309_1 --> 4.14.0-py39hcbf5309_0




conda-4.14.0         | 1.0 MB    |            |   0% 
conda-4.14.0         | 1.0 MB    | 1          |   2% 
conda-4.14.0         | 1.0 MB    | ###        |  31% 
conda-4.14.0

In [4]:
import featuretools as ft
import utils
from utils import load_nyc_taxi_data, compute_features, preview, feature_importances
from sklearn.ensemble import GradientBoostingRegressor
from featuretools.primitives import (Minute, Hour, Day, Week, Month,
                                     Weekday, IsWeekend, Count, Sum, Mean, Median, Std, Min, Max)
import numpy as np
ft.__version__
%load_ext autoreload
%autoreload 2

### Step 1: Download and load the raw data as pandas dataframes

In [5]:
trips, pickup_neighborhoods, dropoff_neighborhoods = load_nyc_taxi_data()
preview(trips, 10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,trip_duration,pickup_neighborhood,dropoff_neighborhood
0,0,2,2016-01-01 00:00:19,2016-01-01 00:06:31,3,1.32,-73.961258,40.796200,-73.950050,40.787312,2,372.0,AH,C
672146,672146,1,2016-04-29 07:01:31,2016-04-29 07:15:46,1,3.30,-73.949951,40.784653,-73.982536,40.755470,1,855.0,C,AA
672147,672147,2,2016-04-29 07:01:43,2016-04-29 07:09:15,1,1.14,-73.967331,40.757370,-73.954277,40.765282,1,452.0,N,K
672148,672148,1,2016-04-29 07:01:46,2016-04-29 07:07:54,1,1.10,-74.003082,40.727509,-73.984703,40.724377,1,368.0,AB,AC
672149,672149,2,2016-04-29 07:01:46,2016-04-29 07:06:48,2,1.40,-73.990158,40.772350,-73.982147,40.759800,1,302.0,AR,AA
672150,672150,1,2016-04-29 07:01:59,2016-04-29 07:07:33,1,1.20,-73.983681,40.746677,-73.971703,40.762463,2,334.0,AO,A
672151,672151,2,2016-04-29 07:02:11,2016-04-29 07:15:24,2,2.13,-73.994209,40.750999,-73.969391,40.761539,1,793.0,D,AK
672152,672152,1,2016-04-29 07:02:11,2016-04-29 07:06:44,1,1.00,-73.983276,40.770985,-73.980110,40.760666,1,273.0,AR,A
672153,672153,2,2016-04-29 07:02:13,2016-04-29 07:08:36,1,1.17,-73.980141,40.743168,-73.983391,40.754665,1,383.0,Y,AA
672154,672154,1,2016-04-29 07:02:16,2016-04-29 07:04:07,1,0.50,-73.965973,40.765381,-73.970558,40.758724,1,111.0,AK,N


The ``trips`` table has the following fields
* ``id`` which uniquely identifies the trip
* ``vendor_id`` is the taxi cab company - in our case study we have data from three different cab companies
* ``pickup_datetime`` the time stamp for pickup
* ``dropoff_datetime`` the time stamp for drop-off
* ``passenger_count`` the number of passengers for the trip
* ``trip_distance`` total distance of the trip in miles 
* ``pickup_longitude`` the longitude for pickup
* ``pickup_latitude`` the latitude for pickup
* ``dropoff_longitude``the longitude of dropoff 
* ``dropoff_latitude`` the latitude of dropoff
* ``payment_type`` a numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided
* ``trip_duration`` this is the duration we would like to predict using other fields 
* ``pickup_neighborhood`` a one or two letter id of the neighboorhood where the trip started
* ``dropoff_neighborhood`` a one or two letter id of the neighboorhood where the trip ended

Step 2: Prepare the Data

Lets create entities and relationships. The three entities in this data are

trips
pickup_neighborhoods
dropoff_neighborhoods
This data has the following relationships

pickup_neighborhoods --> trips (one neighboorhood can have multiple trips that start in it. This means pickup_neighborhoods is the parent_entity and trips is the child entity)
dropoff_neighborhoods --> trips (one neighboorhood can have multiple trips that end in it. This means dropoff_neighborhoods is the parent_entity and trips is the child entity)

In [6]:
entities = {
        "trips": (trips, "id", 'pickup_datetime' ),
        "pickup_neighborhoods": (pickup_neighborhoods, "neighborhood_id"),
        "dropoff_neighborhoods": (dropoff_neighborhoods, "neighborhood_id"),
        }

relationships = [("pickup_neighborhoods", "neighborhood_id", "trips", "pickup_neighborhood"),
                 ("dropoff_neighborhoods", "neighborhood_id", "trips", "dropoff_neighborhood")]

# Q 1.1 Paste in the first 5 Cutoff Times. 1.2 Do these cutoff times make sense? Why did we choose these for the cutoff times?

In [8]:
cutoff_time = trips[['id', 'pickup_datetime']]
cutoff_time = cutoff_time[cutoff_time['pickup_datetime'] > "2016-01-12"]
preview(cutoff_time, 5)

,id,pickup_datetime
56311,56311,2016-01-12 00:00:25
698765,698765,2016-05-03 18:54:53
698766,698766,2016-05-03 18:55:37
698767,698767,2016-05-03 18:55:38
698768,698768,2016-05-03 18:55:49


In [9]:
trans_primitives = [IsWeekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [10]:
print ("Number of features: %d" % len(features))
features

Number of features: 13


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

In [11]:
def compute_features(features, cutoff_time):
    # shuffle so we don't see encoded features in the front or backs

    np.random.shuffle(features)
    feature_matrix = ft.calculate_feature_matrix(features,
                                                 cutoff_time=cutoff_time,
                                                 approximate='36d',
                                                 verbose=True,entities=entities, relationships=relationships)
    print("Finishing computing...")
    feature_matrix, features = ft.encode_features(feature_matrix, features,
                                                  to_encode=["pickup_neighborhood", "dropoff_neighborhood"],
                                                  include_unknown=False)
    return feature_matrix

In [12]:
feature_matrix = compute_features(features, cutoff_time)

Elapsed: 00:07 | Progress: 100%|███████████████████████████████████████████████████████████████████████████████████████
Finishing computing...


In [13]:
preview(feature_matrix, 5)

,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,...,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O,IS_WEEKEND(dropoff_datetime)
id,,,,,,,,,,,,,,,,,,,,,
56311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691285,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691286,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


# Q 2.1 What was the Modeling Score after your last training round? 2.2 Hypothesize on how including more robust features will change the accuracy? # 

In [14]:
# separates the whole feature matrix into train data feature matrix, 
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [15]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925            1.90m
         2           0.4333            1.79m
         3           0.3843            1.78m
         4           0.3446            1.79m
         5           0.3119            1.77m
         6           0.2852            1.76m
         7           0.2634            1.77m
         8           0.2454            1.75m
         9           0.2305            1.75m
        10           0.2183            1.72m
        20           0.1666            1.53m
        30           0.1558            1.34m
        40           0.1514            1.14m
        50           0.1488           55.55s
        60           0.1472           43.79s
        70           0.1458           32.56s
        80           0.1448           21.56s
        90           0.1440           10.70s
       100           0.1433            0.00s


0.7220107526801756

# 3.1 Paste in 5 of the generated features. 3.2 What are these features called and how are they generated?

In [16]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [17]:
print ("Number of features: %d" % len(features))
features

Number of features: 25


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

In [18]:
preview(feature_matrix, 10)

,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,...,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O,IS_WEEKEND(dropoff_datetime)
id,,,,,,,,,,,,,,,,,,,,,
56311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691285,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691286,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
691289,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691290,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
691291,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691292,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


# 4.1What was the Modeling Score after your last training round when including the transform primitives? 4.2 Comment on how the modeling accuracy differs when including the Transform features.

In [19]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [20]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train,y_train)
model.score(X_test,y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925            1.87m
         2           0.4333            1.87m
         3           0.3843            1.79m
         4           0.3446            1.74m
         5           0.3119            1.75m
         6           0.2852            1.71m
         7           0.2634            1.70m
         8           0.2454            1.68m
         9           0.2305            1.65m
        10           0.2183            1.64m
        20           0.1666            1.48m
        30           0.1558            1.29m
        40           0.1514            1.11m
        50           0.1488           54.87s
        60           0.1472           43.19s
        70           0.1458           32.03s
        80           0.1448           21.19s
        90           0.1440           10.52s
       100           0.1433            0.00s


0.7220017570457145

In [21]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]
aggregation_primitives = [Count, Sum, Mean, Median, Std, Max, Min]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=aggregation_primitives,
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [22]:
print ("Number of features: %d" % len(features))
features

Number of features: 63


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: pickup_neighborhoods.COUNT(trips)>,
 <Feature: pickup_neighborhoods.MAX(trips.passenger_count)>

In [23]:
preview(feature_matrix, 10)

,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,...,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O,IS_WEEKEND(dropoff_datetime)
id,,,,,,,,,,,,,,,,,,,,,
56311,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691285,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691286,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691288,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
691289,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691290,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
691291,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
691292,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


# 5.1 What was the Modeling Score after your last training round when including the aggregate transforms? 5.2 How do these aggregate transforms impact performance? How do they impact training time? 

In [24]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [25]:
# note: this may take up to 30 minutes to run
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.4925            1.85m
         2           0.4333            1.89m
         3           0.3843            1.85m
         4           0.3446            1.82m
         5           0.3119            1.82m
         6           0.2852            1.80m
         7           0.2634            1.79m
         8           0.2454            1.78m
         9           0.2305            1.74m
        10           0.2183            1.72m
        20           0.1666            1.51m
        30           0.1558            1.34m
        40           0.1514            1.12m
        50           0.1488           55.24s
        60           0.1472           43.33s
        70           0.1458           32.05s
        80           0.1448           21.26s
        90           0.1440           10.58s
       100           0.1433            0.00s


GradientBoostingRegressor(verbose=True)

In [26]:
model.score(X_test,y_test)

0.7220107526801757

In [27]:
y_pred = model.predict(X_test)
y_pred = np.exp(y_pred) - 1 # undo the log we took earlier
y_pred[5:]

array([ 458.30196275,  467.32756081, 1184.9108786 , ..., 1141.61667765,
       1828.1012845 ,  961.90548057])